In [141]:
import sqlite3
import pandas as pd
import dspy
import dotenv, os
from pydantic import BaseModel, Field
import openai
import pm4py
import ast
from numpy import random
from dspy.evaluate import Evaluate
from collections import defaultdict
import tqdm as notebook_tqdm
import copy
import re
from dspy.teleprompt import BootstrapFewShot
import csv
import phoenix as px
from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from dspy.primitives.assertions import assert_transform_module, backtrack_handler
from chroma_retriever import Chroma
from chromadb.utils import embedding_functions
import numpy as np
import functools
from PY_programs.python_tables import PM_PY_no_deep
from PY_programs.python_simple import PM_PY_simple

# 0.Data loading

In [2]:
INPUT_FILE_NAME = "/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/dataset/Road_Traffic_Fine_Management_Process.xes" #replce with your file path
SQLITE_DB_NAME = "python_test.db" #"my_database.db" #leve as is
LLM_MODEL_TYPE = "gpt-4o"#"gpt-3.5-turbo-0125" #leave as is for gpt 3.5 or change to "gpt-4-turbo" for gpt 4 turbo
SQLITE_DB_EVAL_NAME = "python_eval.db"

In [3]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")

In [165]:
dotenv.load_dotenv()
lm = dspy.LM('openai/gpt-4o', temperature=0.3, max_tokens=4096, stop=None, cache=False)
dspy.settings.configure(lm = lm)

In [52]:
df = pm4py.read_xes(INPUT_FILE_NAME)
df.columns = df.columns.str.replace(':', '_', regex=False)
# replace nan values with 0 in columns "amount", "expense", "paymentAmount", "totalPaymentAmount"
df['amount'] = df['amount'].fillna(0)
df['expense'] = df['expense'].fillna(0)
df['paymentAmount'] = df['paymentAmount'].fillna(0)
df['totalPaymentAmount'] = df['totalPaymentAmount'].fillna(0)
#fturn euros into cents
df['amount'] = df['amount'] * 100
df['amount'] = df['amount'].astype(int)
df['expense'] = df['expense'] * 100
df['expense'] = df['expense'].astype(int)
df['paymentAmount'] = df['paymentAmount'] * 100
df['paymentAmount'] = df['paymentAmount'].astype(int)
df['totalPaymentAmount'] = df['totalPaymentAmount'] * 100
df['totalPaymentAmount'] = df['totalPaymentAmount'].astype(int)

conn = sqlite3.connect(SQLITE_DB_NAME)
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS temp_table;")

print(cur.fetchall())
conn.commit()

df.to_sql("event_log", conn, if_exists="replace", index=False)
cur = conn.cursor()
cur.execute("CREATE INDEX idx_case_concept_name_event_log ON event_log(case_concept_name)")
conn.commit()

cur.execute("ALTER TABLE event_log ADD COLUMN idx INTEGER;")
conn.commit()
cur.execute("""UPDATE event_log SET idx = (
    SELECT rowid FROM (
        SELECT ROW_NUMBER() OVER (ORDER BY case_concept_name, time_timestamp) as seq_num, rowid
        FROM event_log
    ) temp WHERE temp.rowid = event_log.rowid
);""")
conn.commit()
cur.execute("CREATE INDEX idx_event_log_idx ON event_log(idx);")
conn.commit()
cur.execute("VACUUM;")
cur.execute("ANALYZE;")
conn.commit()
cur.close()

parsing log, completed traces ::   0%|          | 0/150370 [00:00<?, ?it/s]

[]


In [7]:
phoenix_session = px.launch_app()
endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

E0000 00:00:1735216702.722517 13503914 chttp2_server.cc:1177] UNKNOWN:No address added out of total 1 resolved for '[::]:4317' {created_time:"2024-12-26T13:38:22.722512+01:00", children:[UNKNOWN:Failed to add any wildcard listeners {created_time:"2024-12-26T13:38:22.722499+01:00", children:[UNKNOWN:Unable to configure socket {created_time:"2024-12-26T13:38:22.722471+01:00", fd:95, children:[UNKNOWN:bind: Address already in use (48) {created_time:"2024-12-26T13:38:22.722446+01:00"}]}, UNKNOWN:Unable to configure socket {fd:95, created_time:"2024-12-26T13:38:22.722497+01:00", children:[UNKNOWN:bind: Address already in use (48) {created_time:"2024-12-26T13:38:22.72249+01:00"}]}]}]}
ERROR:    Traceback (most recent call last):
  File "/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/dspy_v2/lib/python3.12/site-packages/starlette/routing.py", line 732, in lifespan
    async with self.lifespan_context(app) as maybe_state:
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [171]:
col_desc = """- 'amount' (int): The amount due to be paid for the fine (including the penalty amount in case it is added). There are no nan values in this column.
- 'org_resource' (int): A numeric code indicating the employee who handled the case.
- 'dismissal' (string): A flag indicating whether and by whom the fine is dismissed. It is initialized to NIL. We know the meaning of:  
        'G': dismissed by the judge
        '#': dismissed by the prefecture
        NIL: not dismissed, i.e., to be paid.
        There are several other values used for which we do not know the semantics.
- 'vehicleClass' (string): A flag indicating the kind of vehicle driven or owned by the offender. The semantics of the values is unknown.
- 'totalPaymentAmount' (int): The cumulative amount paid by the offender. It is always initialized to 0. There are no nan values in this column.
- 'lifecycle_transition' (string): the transition of the activity (complete, start, etc.)
- 'article' (string): The number of the article of the Italian roadtraffic law that is violated by the offender (e.g., article 157 refers to stopping and parking vehicles).
- 'points' (float): Penalty points deducted from the driving license. In Italy, each driver starts with 20 points on their license and may loose points for each offence, based on the gravity.
- 'expense' (int): The additional amount due to be paid for postal expenses. There are no nan values in this column.
- 'notificationType' (string): A flag indicating to whom the fine refers. 'P': car owner, 'C': car driver.
- 'lastSent' (datetime): N/A
- 'paymentAmount' (int): The amount paid by the offender in one transaction. There are no nan values in this column.
- 'matricola' (string): N/A (Probably refers to the matriculation number of the car.)
- 'concept_name' (string): the activity/ event type name
        Activity Description, column: 'concept:name':
            > 'Create Fine': The initial creation of the fine in the information system. It initializes event log attributes amount, dismissal, points and totalPaymentAmount.
            > 'Send Fine': A notification about the fine is sent by post to the offender.
            > 'Insert Fine Notification': The notification is received by the offender.
            > 'Add penalty': An additional penalty is applied.
            > 'Payment': A payment made by the offender is registered.
            > 'Send for Credit Collection': Unpaid fines are sent for credit collection. A separate process is started by a collection agency to collect the money of the unpaid fines.
            > 'Insert Date Appeal to Prefecture': The offender appeals against the fine to the prefecture. A prefecture in Italy is an administrative body representing the national government in each province.
            > 'Send Appeal to Prefecture': The appeal is sent to the prefecture by the local police.
            > 'Receive Result Appeal from Prefecture': The local police receives the result of the appeal. If the prefecture dismisses the fine, the appeal is deemed accepted, and the obligation to pay the fine is cancelled. In this case, there is no need for the police to receive the result from the prefecture (Receive Result Appeal from Prefecture) and notify the offender (Notify Result Appeal to Offender).
            > 'Notify Result Appeal to Offender': The local police informs the offender of the appeal result. 
            > 'Appeal to Judge': The offender appeals against the fine to a judge.
        IMPORTANT: The last event in a case can be arbitrary. There is no guarantee that the last event is 'Send Fine' or 'Payment'. The last event can be any event in the log."""

In [172]:
# initialize the chroma retriever
rm = Chroma(sentence_transformer_ef = sentence_transformer_ef , documentation = col_desc) # for python, use standard cols

["- 'amount' (int): The amount due to be paid for the fine (including the penalty amount in case it is added). There are no nan values in this column.", "- 'org_resource' (int): A numeric code indicating the employee who handled the case.", "- 'dismissal' (string): A flag indicating whether and by whom the fine is dismissed. It is initialized to NIL. We know the meaning of:  \n        'G': dismissed by the judge\n        '#': dismissed by the prefecture\n        NIL: not dismissed, i.e., to be paid.\n        There are several other values used for which we do not know the semantics.", "- 'vehicleClass' (string): A flag indicating the kind of vehicle driven or owned by the offender. The semantics of the values is unknown.", "- 'totalPaymentAmount' (int): The cumulative amount paid by the offender. It is always initialized to 0. There are no nan values in this column.", "- 'lifecycle_transition' (string): the transition of the activity (complete, start, etc.)", "- 'article' (string): The

In [173]:
rm.add_new("- 'obligation_topay_cancelled' (int): A boolean indicator (stored as an integer) showing whether the obligation to pay the fine is cancelled due to the time difference between 'Create Fine' and 'Send Fine' being more than 90 days. 1 for cancelled, 0 otherwise. This value is consistent across all rows pertaining to the same case.")

# Saving Class

In [10]:
def save_report_PM(outputs,scores,filename, program, conn = conn):
    inst_lst = []
    col_lst = []
    dfn_lst = []
    example_lst = []
    rationale_lst = []
    description_lst = []
    for i in range(len(outputs)):
        inst_lst.append(outputs[i][0].instruction)
        col_lst.append(outputs[i][0].column)
        dfn_lst.append(outputs[i][0].df_name)
        example_lst.append(outputs[i][0].example)
        try:
            rationale_lst.append(outputs[i][1].rationale)
        except:
            rationale_lst.append("No rationale")
        try:
            description_lst.append(outputs[i][1].description)
        except:
            description_lst.append("No description")
    code, errors, descriptions = program.get_history()
    code_1 = []
    code_2 = []
    code_3 = []
    for i in inst_lst:
        try:
            code_1.append(code[i][0])
        except:
            code_1.append("No code")
        try:
            code_2.append(code[i][1])
        except:
            code_2.append("No code")
        try:
            code_3.append(code[i][2])
        except:
            code_3.append("No code")
    error_lst = []
    for i in inst_lst:
        error_lst.append(errors[i])
    query = 'SELECT * FROM event_log'
    du = pd.read_sql_query(query, conn, parse_dates = ['time_timestamp'])
    if "duration" in du.columns:
        du['duration'] = pd.to_timedelta(du['duration'], unit = "s")
    for cols in du.columns:
        if du[cols].isin([0,1]).all() and not cols.endswith("_count"):
                du[cols] = du[cols].astype(bool)
    du_sorted = du.sort_values(by=['case_concept_name', 'time_timestamp'])
    g_dfs = []
    for i in col_lst:
        try:
            pred_col = du_sorted[i]
        except:
            pred_col = "error"
        g_dfs.append(pred_col)
    lst_non_equal = []
    for i in range(len(example_lst)):
        try:
            non_equal_elements = example_lst[i] != g_dfs[i]
            lst_non_equal.append(non_equal_elements.sum())
        except:
            lst_non_equal.append("error")
    df_merged = pd.DataFrame({'Instruction': inst_lst, 'Column': col_lst, 'df_name': dfn_lst, 'Example': example_lst, 'Rationale': rationale_lst, 'Description': description_lst, "PM_Metric": scores, 'Code 1': code_1, 'Code 2': code_2, 'Code 3': code_3, "errors": error_lst, 'pred_col': g_dfs, "num_rows_different": lst_non_equal})
    df_merged.to_csv(f"{filename}.csv", index=False, quoting=csv.QUOTE_ALL, escapechar='\\')
    return df_merged

# Metric

In [174]:
def PM_Metric(example, prediction, trace = None, conn=conn):
    col = example.column
    question = example.instruction
    df_name = example.df_name
    gold_col = example.example
    
    query = 'SELECT * FROM event_log'
    try:
        tp = pd.read_sql_query(query, conn, parse_dates = ['time_timestamp'])

        if "duration" in tp.columns:
            tp['duration'] = pd.to_timedelta(tp['duration'], unit = "s")
    except Exception as e:
        print(f"Error executing SQL query: {str(e)}")
        return False
    for cols in tp.columns:
        if tp[cols].isin([0,1]).all() and not cols.endswith("_count"):
            tp[cols] = tp[cols].astype(bool)
    
    tp_sorted = tp.sort_values(by=['case_concept_name', 'time_timestamp'])

    # if gold_col is of type timedelta, convert to int (seconds)
    #if gold_col.dtype == '<m8[ns]':
    #    gold_col = gold_col.dt.total_seconds().astype(int)

    try:
        
        pred_col = tp_sorted[col]
    except KeyError:
        print(f"Eval could not retrieve col: {col}")
        return False

    # Use .equals to determine if the Series are the same
    if pred_col.equals(gold_col):
        if len(pred_col) == len(gold_col):
            print("EVAL TRUE")
            
        
            return True
        else:
            print("EVAL FALSE")
            return False
    else:

        print("EVAL FALSE")
        print("Type of pred_col", pred_col.dtype)
        print("Type of gold_col", gold_col.dtype)
        # number of rows that are different
        try:
            non_equal_elements = pred_col != gold_col
            print("Number of rows different", non_equal_elements.sum())
        except Exception as e:
            print(f"Error in comparison: {str(e)}")
            return False
        return False

In [12]:
def PM_Metric_training(example, prediction, trace = None, df=df):
    conn = sqlite3.connect("python_test.db")
    col = example.column
    question = example.instruction
    df_name = example.df_name
    gold_col = example.example
    
    query = 'SELECT * FROM event_log'
    try:
        dp = pd.read_sql_query(query, conn, parse_dates = ['time_timestamp'])

        if "duration" in dp.columns:
            dp['duration'] = pd.to_timedelta(dp['duration'], unit = "s")
    except Exception as e:
        print(f"Error executing SQL query: {str(e)}")
        return False
    for cols in dp.columns:
        if dp[cols].isin([0,1]).all() and not cols.endswith("_count"):
            dp[cols] = dp[cols].astype(bool)
    
    dp_sorted = dp.sort_values(by=['case_concept_name', 'time_timestamp'])

    cur = conn.cursor()
    

    df.to_sql("event_log", conn, if_exists="replace", index=False)
    cur = conn.cursor()
    cur.execute("CREATE INDEX idx_case_concept_name_event_log ON event_log(case_concept_name)")
    conn.commit()

    cur.execute("ALTER TABLE event_log ADD COLUMN idx INTEGER;")
    conn.commit()
    cur.execute("""UPDATE event_log SET idx = (
        SELECT rowid FROM (
            SELECT ROW_NUMBER() OVER (ORDER BY case_concept_name, time_timestamp) as seq_num, rowid
            FROM event_log
        ) temp WHERE temp.rowid = event_log.rowid
    );""")
    conn.commit()
    cur.execute("CREATE INDEX idx_event_log_idx ON event_log(idx);")
    conn.commit()
    cur.execute("VACUUM;")
    cur.execute("ANALYZE;")
    conn.commit()
    cur.close()
    
    # if gold_col.dtype == '<m8[ns]':
    #     gold_col = gold_col.dt.total_seconds().astype(int)

    try:
        pred_col = dp_sorted[col]
    except KeyError:
        print(f"Eval could not retrieve col: {col}")
        return False

    # Use .equals to determine if the Series are the same
    if pred_col.equals(gold_col):
        print("EVAL TRUE")
        return True
    print("EVAL FALSE")
    print("Type of pred_col", type(pred_col[0]))
    print("Type of gold_col", type(gold_col[0]))
    # number of rows that are different
    non_equal_elements = pred_col != gold_col
    print("Number of rows different", non_equal_elements.sum())
    return False

# Loading Dataset

In [13]:
gold = pd.read_csv("/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/benchmark/PM_EVALQUESTIONS_DF4.csv")
gold['time_timestamp'] = pd.to_datetime(gold['time_timestamp'])
gold['duration'] = pd.to_timedelta(gold['duration'], unit= "s")
gold_sorted = gold.sort_values(by=['case_concept_name', 'time_timestamp'])
gold_sorted = gold_sorted.reset_index(drop=True)

In [14]:
gold_sorted.groupby("case_concept_name")["paid_without_obligation"].value_counts()

case_concept_name  paid_without_obligation
A1                 False                      2
A100               False                      5
A10000             True                       5
A10001             False                      6
A10004             False                      5
                                             ..
V9995              False                      5
V9996              False                      3
V9997              False                      5
V9998              False                      5
V9999              False                      5
Name: count, Length: 150370, dtype: int64

In [15]:
gold_sorted.head()

,amount,org_resource,dismissal,concept_name,vehicleClass,totalPaymentAmount,lifecycle_transition,time_timestamp,article,points,...,underpaid,credit_collected_AND_fully_paid,dismissed_AND_fully_paid,overpaid_amount,underpaid_amount,part_paid,unresolved,paid_without_obligation,time_timestamp_beginn,time_timestamp_end
0,3500,561.0,NIL,Create Fine,A,0,complete,2006-07-24 00:00:00+00:00,157.0,0.0,...,True,False,False,0,4600,False,True,False,2006-07-24 00:00:00+00:00,2006-12-05 00:00:00+00:00
1,0,NaN,NaN,Send Fine,NaN,0,complete,2006-12-05 00:00:00+00:00,NaN,NaN,...,True,False,False,0,4600,False,True,False,2006-07-24 00:00:00+00:00,2006-12-05 00:00:00+00:00
2,3500,561.0,NIL,Create Fine,A,0,complete,2006-08-02 00:00:00+00:00,157.0,0.0,...,True,False,False,0,8250,False,False,False,2006-08-02 00:00:00+00:00,2009-03-30 00:00:00+00:00
3,0,NaN,NaN,Send Fine,NaN,0,complete,2006-12-12 00:00:00+00:00,NaN,NaN,...,True,False,False,0,8250,False,False,False,2006-08-02 00:00:00+00:00,2009-03-30 00:00:00+00:00
4,0,NaN,NaN,Insert Fine Notification,NaN,0,complete,2007-01-15 00:00:00+00:00,NaN,NaN,...,True,False,False,0,8250,False,False,False,2006-08-02 00:00:00+00:00,2009-03-30 00:00:00+00:00


In [16]:
qa = pd.read_csv("/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/benchmark/PM_EVALQUESTIONS_final.csv")
qa.head()

,Definition,Column,Dependent On New Col,DF
0,"Create column called ""dismissed_by_prefecture""...",dismissed_by_prefecture,NaN,1
1,"Create column called ""dismissed_by_judge"", def...",dismissed_by_judge,NaN,1
2,"Create a column called ""maxtotalPaymentAmount""...",maxtotalPaymentAmount,NaN,1
3,"Create a column called ""duration"" which takes ...",duration,NaN,1
4,"Create a column called ""event_count"", type int...",event_count,NaN,1


In [17]:
for i in range(len(qa)):
    if qa["Column"][i] == "obligation_topay_cancelled":
        print(i)
    elif qa["Column"][i] == "obligation_to_pay_cancelled":
        print(i)

10


In [18]:
qa["Column"][10]

'obligation_topay_cancelled'

In [19]:
dataset = []

for instruction, col, dep, df_name in qa.values:
    dataset.append(dspy.Example(instruction = instruction, column = col, df_name = df_name, example = gold_sorted[col]).with_inputs("instruction"))


In [20]:
# the following columns will be in the trainset: duration, event_count, amount_min, amount_last, obligation_topay_cancelled, dismissed_by_other, appeal_to_judgeorprefecture, add_penalty_count, send_fine_count, payment_count, insert_fine_notification_count, send_for_credit_collection_count, insert_date_appeal_to_prefecture_count, send_appeal_to_prefecture_count, receive_result_appeal_from_prefecture_count, notify_result_appeal_to_offender_count, time_timestamp_beginn, time_timestamp_end, appeal_to_judge_count

# use those columns for train set

trainset = []

cols_in_trainset = ['duration', 'event_count', 'amount_min', 'obligation_topay_cancelled', 'dismissed_by_other', 'appeal_to_judgeorprefecture', 'add_penalty_count', 'send_fine_count', 'payment_count', 'insert_fine_notification_count', 'send_for_credit_collection_count', 'insert_date_appeal_to_prefecture_count', 'send_appeal_to_prefecture_count', 'receive_result_appeal_from_prefecture_count', 'notify_result_appeal_to_offender_count', 'time_timestamp_beginn', 'time_timestamp_end', 'appeal_to_judge_count']

for i in range(len(dataset)):
    if dataset[i]["column"] in cols_in_trainset:
        trainset.append(dataset[i])

print(len(trainset))

# create test set which uses the remaining columns
trainset[0], trainset[3] = trainset[3], trainset[0]

testset = []

for i in range(len(dataset)):
    if dataset[i] not in trainset:
        testset.append(dataset[i])

print(len(testset))


18
25


# Evaluation

**'openai/gpt-4o', temperature=0.3, max_tokens=4096,**

- PM_PY_no_deep (with assertions): 68%, 13m NAME: PY_testset_uncompiled
- PM_PY_no_deep_fewshot_12 (with assertions): 100%, 10m 51s NAME: PY_testset_add_fewshot12_final
- PM_PY_no_deep_fewshot_12 (no assertions): 92%, 9m 50s NAME: PY_testset_add_fewshot12_no_assertion
- PM_PY_simple (with assertions): 28.0%, 29m 30s, NAME: PY_testset_simple (9/25 cols correct, descriptions wrong)
- PM_PY_simple (no assertions): 0%, 8m 30s, NAME: PY_testset_simple_no_assertion (6/25 cols correct, descriptions wrong)


**'openai/gpt-4o-mini', temperature=0.3, max_tokens=4096,**

- PM_PY_no_deep (with assertions): 36.0%, 22m 54s Name: PM_PY_testset_mini_uncompiled
- PM_PY_no_deep (no assertions): 0%, 5m, 7s (because the column descriptions could not be fed into the chroma retriever), in terms of cols (9/25 cols were correct) Name: PM_PY_testset_mini_uncompiled_no_assertion
- PM_PY_no_deep_fewshot_12 (with assertions): 92.0% 14m 29s, Name : PY_testset_mini_fewshot12
- PM_PY_no_deep_fewshot_12 (no assertions): 92.0% 10m 20s, Name: PY_testset_mini_fewshot12_no_assertion (score is super high, maybe run again)*
- PM_PY_simple (with assertions): 24.0%, 37m Name: PY_testset_mini_simple (a few descriptions were incorrect (10/25 cols ))
- PM_PY_simple (no assertions): 0.0%, 8m 52s, Name: PY_testset_mini_simple_no_assertion (9/25 col were correct, descriptions wrong)

for the evaluation on the trainset, we must make sure that the columns that require them are present. Therefore, we make two databases. One that is only the vanilla event log, and the other which includes the columns from the trainset.

In [128]:
uncompiled_py = assert_transform_module(PM_PY_no_deep(conn_path= SQLITE_DB_EVAL_NAME, rm=rm, training_mode=False ), functools.partial(backtrack_handler, max_backtracks=4))
#uncompiled_py.load("/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/Optimized_prompts/python/py_add_fewshot_12.json")
#uncompiled_py._compiled = True

In [137]:
uncompiled_py_no_assert = PM_PY_no_deep(conn_path= SQLITE_DB_EVAL_NAME, rm=rm, training_mode=False )
#compiled_no_assert.load("/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/Optimized_prompts/python/py_add_fewshot_12.json")
#compiled_no_assert._compiled = True

In [166]:
un_py_simple = assert_transform_module(PM_PY_simple(conn_path= SQLITE_DB_EVAL_NAME, rm=rm), functools.partial(backtrack_handler, max_backtracks=4))

In [175]:
un_py_simple_no_assert = PM_PY_simple(conn_path= SQLITE_DB_EVAL_NAME, rm=rm)

In [160]:
df_t = pm4py.read_xes(INPUT_FILE_NAME)
df_t.columns = df_t.columns.str.replace(':', '_', regex=False)
# replace nan values with 0 in columns "amount", "expense", "paymentAmount", "totalPaymentAmount"
df_t['amount'] = df_t['amount'].fillna(0)
df_t['expense'] = df_t['expense'].fillna(0)
df_t['paymentAmount'] = df_t['paymentAmount'].fillna(0)
df_t['totalPaymentAmount'] = df_t['totalPaymentAmount'].fillna(0)
#fturn euros into cents
df_t['amount'] = df_t['amount'] * 100
df_t['amount'] = df_t['amount'].astype(int)
df_t['expense'] = df_t['expense'] * 100
df_t['expense'] = df_t['expense'].astype(int)
df_t['paymentAmount'] = df_t['paymentAmount'] * 100
df_t['paymentAmount'] = df_t['paymentAmount'].astype(int)
df_t['totalPaymentAmount'] = df_t['totalPaymentAmount'] * 100
df_t['totalPaymentAmount'] = df_t['totalPaymentAmount'].astype(int)
df_t['obligation_topay_cancelled'] = gold_sorted['obligation_topay_cancelled']

conn = sqlite3.connect(SQLITE_DB_EVAL_NAME)
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS temp_table;")

print(cur.fetchall())
conn.commit()

df_t.to_sql("event_log", conn, if_exists="replace", index=False)
cur = conn.cursor()
cur.execute("CREATE INDEX idx_case_concept_name_event_log ON event_log(case_concept_name)")
conn.commit()

cur.execute("ALTER TABLE event_log ADD COLUMN idx INTEGER;")
conn.commit()
cur.execute("""UPDATE event_log SET idx = (
    SELECT rowid FROM (
        SELECT ROW_NUMBER() OVER (ORDER BY case_concept_name, time_timestamp) as seq_num, rowid
        FROM event_log
    ) temp WHERE temp.rowid = event_log.rowid
);""")
conn.commit()
cur.execute("CREATE INDEX idx_event_log_idx ON event_log(idx);")
conn.commit()
cur.execute("VACUUM;")
cur.execute("ANALYZE;")
conn.commit()
cur.close()

parsing log, completed traces ::   0%|          | 0/150370 [00:00<?, ?it/s]

[]


In [37]:
len(testset)

25

In [57]:
testset[24:]

[Example({'instruction': 'Create a column called "paid_without_obligation" which is a boolean on a per case basis. TRUE if the column "obligation_topay_cancelled" == TRUE AND the column "fully_paid" == TRUE. If else "paid_without_obligation" == FALSE. Makse sure all values for "paid_without_obligation" are the same across all rows of a case.', 'column': 'paid_without_obligation', 'df_name': 4, 'example': 0         False
 1         False
 2         False
 3         False
 4         False
           ...  
 561465    False
 561466    False
 561467    False
 561468    False
 561469    False
 Name: paid_without_obligation, Length: 561470, dtype: bool}) (input_keys={'instruction'})]

In [176]:
evaluate = Evaluate(devset=testset, metric= PM_Metric, num_threads = 1, display_progress= True, display_table = len(testset), return_outputs=True, provide_traceback=True, max_errors=30)

In [177]:
score, outputs, scores = evaluate(program = un_py_simple_no_assert, return_all_scores= True)

  0%|          | 0/25 [00:00<?, ?it/s]length of dp 561470
column type of new column bool
pre_float_check True
pre_distinc_value_check True


2024/12/27 13:04:34 INFO dspy.primitives.assertions: SuggestionFailed: Column description must be short and less than 350 characters
2024/12/27 13:04:34 ERROR dspy.utils.parallelizer: Error processing item Example({'instruction': 'Create column called "dismissed_by_prefecture", defined as a boolean. True if any of the values of a case in column dismissal = "#" , if not False. Apply the same value across all rows of each case', 'column': 'dismissed_by_prefecture', 'df_name': 1, 'example': 0         False
1         False
2         False
3         False
4         False
          ...  
561465    False
561466    False
561467    False
561468    False
561469    False
Name: dismissed_by_prefecture, Length: 561470, dtype: bool}) (input_keys={'instruction'}): Column description must be short and less than 350 characters
Stack trace:
Traceback (most recent call last):
  File "/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/dspy_v2/lib/python3.12/site-packages/dspy/utils/parallelizer.py", line 47

Average Metric: 0.00 / 25 (0.0%):   4%|▍         | 1/25 [00:23<09:31, 23.82s/it]

2024/12/27 13:04:39 INFO dspy.primitives.assertions: SuggestionFailed: Error executing codename 'dp' is not defined
Stack trace:
Line 3: dp['dismissed_by_judge'] = dp.groupby('case_concept_name')['dismissal'].transform(lambda x: 'G' in x.values)
2024/12/27 13:04:39 ERROR dspy.utils.parallelizer: Error processing item Example({'instruction': 'Create column called "dismissed_by_judge", defined as a boolean. True if any of the values of a case in column dismissal = "G" , if not False. Apply the same value across all rows of each case', 'column': 'dismissed_by_judge', 'df_name': 1, 'example': 0         False
1         False
2         False
3         False
4         False
          ...  
561465    False
561466    False
561467    False
561468    False
561469    False
Name: dismissed_by_judge, Length: 561470, dtype: bool}) (input_keys={'instruction'}): Error executing codename 'dp' is not defined
Stack trace:
Line 3: dp['dismissed_by_judge'] = dp.groupby('case_concept_name')['dismissal'].tran

Average Metric: 0.00 / 25 (0.0%):   8%|▊         | 2/25 [00:28<04:52, 12.72s/it]

...

Average Metric: 0.00 / 25 (0.0%):  96%|█████████▌| 24/25 [07:56<00:29, 29.96s/it]

2024/12/27 13:12:42 INFO dspy.primitives.assertions: SuggestionFailed: Do not use .apply and lambda functions to create the new column, as those will result in nan values when grouping by case (subsequent column dtype will be float64 or object). Instead use boolean operators first and subsequently group by case, then counring using for example value_counts() or methods that are more robust in avoiding nan values. (fillna(0).astype(int) will most likely also be incorrect.)
2024/12/27 13:12:42 ERROR dspy.utils.parallelizer: Error processing item Example({'instruction': 'Create a column called "paid_without_obligation" which is a boolean on a per case basis. TRUE if the column "obligation_topay_cancelled" == TRUE AND the column "fully_paid" == TRUE. If else "paid_without_obligation" == FALSE. Makse sure all values for "paid_without_obligation" are the same across all rows of a case.', 'column': 'paid_without_obligation', 'df_name': 4, 'example': 0         False
1         False
2         F

length of dp 561470
column type of new column object
pre_float_check False
Average Metric: 0.00 / 25 (0.0%): 100%|██████████| 25/25 [08:31<00:00, 20.46s/it]

2024/12/27 13:12:42 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 25 (0.0%)


,instruction,column,df_name,example,PM_Metric
0,"Create column called ""dismissed_by_prefecture"", defined as a boole...",dismissed_by_prefecture,1,0 False 1 False 2 False 3 False 4 False ... 561465 False 561466 Fa...,
1,"Create column called ""dismissed_by_judge"", defined as a boolean. T...",dismissed_by_judge,1,0 False 1 False 2 False 3 False 4 False ... 561465 False 561466 Fa...,
2,"Create a column called ""maxtotalPaymentAmount"", type int, which fo...",maxtotalPaymentAmount,1,0 0 1 0 2 0 3 0 4 0 .. 561465 0 561466 0 561467 0 561468 0 561469 ...,
3,"Create a column called ""expense_sum"", defined as type int, which s...",expense_sum,1,0 1100 1 1100 2 1100 3 1100 4 1100 ... 561465 1516 561466 1516 561...,
4,"Create a column called ""amount_last"", defined as type int, which t...",amount_last,1,0 3500 1 3500 2 7150 3 7150 4 7150 ... 561465 26200 561466 26200 5...,
5,"Create a column called ""dismissed"" which is a boolean, TRUE if any...",dismissed,1,0 False 1 False 2 False 3 False 4 False ... 561465 False 561466 Fa...,
6,"Create a column called ""credit_collected"" which is a boolean. True...",credit_collected,1,0 False 1 False 2 True 3 True 4 True ... 561465 True 561466 True 5...,
7,"Create a column called ""penalty_added"" which is a boolean, TRUE if...",penalty_added,1,0 False 1 False 2 True 3 True 4 True ... 561465 True 561466 True 5...,
8,"Create a column called ""appealed_to_judge"" which is a boolean. TRU...",appealed_to_judge,1,0 False 1 False 2 False 3 False 4 False ... 561465 False 561466 Fa...,
9,"Create a column called ""appealed_to_prefecture"" which is a boolean...",appealed_to_prefecture,1,0 False 1 False 2 False 3 False 4 False ... 561465 False 561466 Fa...,


In [178]:
df_merged = save_report_PM(outputs,scores,"/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/Results_PY/PY_testset_simple_no_assertion", un_py_simple_no_assert, conn = conn)

In [170]:
# reset DB and class for another evaluation

conn = sqlite3.connect(SQLITE_DB_EVAL_NAME)
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS temp_table;")

print(cur.fetchall())
conn.commit()

df_t.to_sql("event_log", conn, if_exists="replace", index=False)
cur = conn.cursor()
cur.execute("CREATE INDEX idx_case_concept_name_event_log ON event_log(case_concept_name)")
conn.commit()

cur.execute("ALTER TABLE event_log ADD COLUMN idx INTEGER;")
conn.commit()
cur.execute("""UPDATE event_log SET idx = (
    SELECT rowid FROM (
        SELECT ROW_NUMBER() OVER (ORDER BY case_concept_name, time_timestamp) as seq_num, rowid
        FROM event_log
    ) temp WHERE temp.rowid = event_log.rowid
);""")
conn.commit()
cur.execute("CREATE INDEX idx_event_log_idx ON event_log(idx);")
conn.commit()
cur.execute("VACUUM;")
cur.execute("ANALYZE;")
conn.commit()
cur.close()

[]


# Optimization 

In [25]:
py_special = PM_PY_no_deep(conn_path = SQLITE_DB_NAME, rm=rm, training_mode=True).activate_assertions()
py_special.load("/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/Optimized_prompts/python/py_fewshot_12 copy 2.json")
py_special._compiled = False

In [26]:
uncompiled_py = PM_PY_no_deep(conn_path = SQLITE_DB_NAME, rm=rm).activate_assertions()

In [26]:
# reset DB and class for another evaluation

conn = sqlite3.connect(SQLITE_DB_NAME)
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS temp_table;")

print(cur.fetchall())
conn.commit()

df.to_sql("event_log", conn, if_exists="replace", index=False)
cur = conn.cursor()
cur.execute("CREATE INDEX idx_case_concept_name_event_log ON event_log(case_concept_name)")
conn.commit()

cur.execute("ALTER TABLE event_log ADD COLUMN idx INTEGER;")
conn.commit()
cur.execute("""UPDATE event_log SET idx = (
    SELECT rowid FROM (
        SELECT ROW_NUMBER() OVER (ORDER BY case_concept_name, time_timestamp) as seq_num, rowid
        FROM event_log
    ) temp WHERE temp.rowid = event_log.rowid
);""")
conn.commit()
cur.execute("CREATE INDEX idx_event_log_idx ON event_log(idx);")
conn.commit()
cur.execute("VACUUM;")
cur.execute("ANALYZE;")
conn.commit()
cur.close()

[]


In [27]:
len(trainset)

18

In [28]:
bootstrap_fewshot = BootstrapFewShot(
    metric = PM_Metric_training,
    max_bootstrapped_demos = 3,
    max_labeled_demos = 0,
    max_rounds = 3,
    max_errors = 2)
py_compiled_special = bootstrap_fewshot.compile(student = py_special, trainset = trainset)

  0%|          | 0/18 [00:00<?, ?it/s]2024/12/26 13:48:36 INFO dspy.primitives.assertions: SuggestionFailed: Error executing codename 'dp' is not defined
Stack trace:
Line 3: case_durations = dp[dp['concept_name'].isin(['Create Fine', 'Send Fine'])]
2024/12/26 13:48:45 INFO dspy.primitives.assertions: SuggestionFailed: Error executing codesingle positional indexer is out-of-bounds
Stack trace:
Line 14: timedeltas = case_durations.groupby('case_concept_name').apply(
Line 15:     lambda x: (x[x['concept_name'] == 'Send Fine']['time_timestamp'].iloc[0] -
2024/12/26 13:49:21 INFO dspy.primitives.assertions: SuggestionFailed: Error executing codename 'timedeltas' is not defined
Stack trace:
Line 24: dp['obligation_topay_cancelled'] = dp['case_concept_name'].map(lambda x: timedeltas.get(x, 0) > 90)
Line 24: dp['obligation_topay_cancelled'] = dp['case_concept_name'].map(lambda x: timedeltas.get(x, 0) > 90)
2024/12/26 13:49:22 INFO dspy.primitives.assertions: SuggestionFailed: New Column was n

length of dp 561470
pre_float_check True
pre_distinc_value_check True
Eval could not retrieve col: obligation_topay_cancelled


2024/12/26 13:49:38 INFO dspy.primitives.assertions: SuggestionFailed: Error executing codename 'dp' is not defined
Stack trace:
Line 3: case_durations = dp[dp['concept_name'].isin(['Create Fine', 'Send Fine'])].groupby(['case_concept_name', 'concept_name'])['time_timestamp'].min().unstack()


length of dp 561470
column type of new column bool
pre_float_check True
pre_distinc_value_check True


  6%|▌         | 1/18 [01:27<24:53, 87.84s/it]

...

EVAL TRUE


2024/12/26 13:50:26 INFO dspy.primitives.assertions: SuggestionFailed: Error executing codename 'dp' is not defined
Stack trace:
Line 3: amount_min_per_case = dp[dp['amount'] > 0].groupby('case_concept_name')['amount'].min().reset_index()


length of dp 561470
column type of new column int64
pre_float_check True
pre_distinc_value_check True


 17%|█▋        | 3/18 [02:14<11:11, 44.75s/it]

EVAL TRUE
Bootstrapped 3 full traces after 3 examples for up to 3 rounds, amounting to 4 attempts.


In [29]:
py_compiled_special.save("/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/Optimized_prompts/python/py_add_fewshot_3.json")

In [35]:
py_compiled_special_2.training_mode

False

In [31]:
bootstrap_fewshot = BootstrapFewShot(
    metric = PM_Metric_training,
    max_bootstrapped_demos = 12,
    max_labeled_demos = 0,
    max_rounds = 3,
    max_errors = 3)
py_compiled_special_2 = bootstrap_fewshot.compile(student = py_special, teacher=py_compiled_special, trainset = trainset)

  0%|          | 0/18 [00:00<?, ?it/s]

length of dp 561470
column type of new column bool
pre_float_check True
pre_distinc_value_check True


  6%|▌         | 1/18 [00:18<05:20, 18.86s/it]

EVAL TRUE
length of dp 561470
column type of new column int64
pre_float_check True
pre_distinc_value_check True


 11%|█         | 2/18 [00:36<04:47, 17.99s/it]

...

EVAL TRUE
length of dp 561470
column type of new column int64
pre_float_check True
pre_distinc_value_check True


 61%|██████    | 11/18 [03:26<02:09, 18.47s/it]

EVAL TRUE
length of dp 561470
column type of new column int64
pre_float_check True
pre_distinc_value_check True


 67%|██████▋   | 12/18 [03:44<01:52, 18.74s/it]

EVAL TRUE
Bootstrapped 12 full traces after 12 examples for up to 3 rounds, amounting to 12 attempts.


In [32]:
py_compiled_special_2.save("/Users/sulzair/Documents/Bachelor Thesis/dspy_v2/Optimized_prompts/python/py_add_fewshot_12.json")